## Importar Librerías

In [1]:
import camelot
import pandas as pd
from datetime import datetime as dt
import glob

## Utilidades

In [2]:
def read_pdf(input_path):
    # Lee los archivos pdf del path (correspondientes a la fecha de descarga), y crea diccionario con respectivos dataframes
    all_files = pd.Series(glob.glob(input_path + "/*.pdf"))
    d = {}
   
    for i in range(0,len(all_files)):
        file = all_files[i]
        index_from = file.rfind("\\") + 1
        
        if 'Diciembre' in file:
            d[str(file[index_from:-4])] = camelot.read_pdf(file, pages='5,6,7,8,9,10,11')
        else:
            d[str(file[index_from:-4])] = camelot.read_pdf(file, pages='2,3,4,5,6,7,8')
       
    return d

def num_format(df,col):
    df[col] = df[col].replace('','0').str.replace('-','0').str.replace('(','-').str.replace(')','').str.replace('.','').str.replace(' ','0').astype(float)
    return df

def cols_in_df(df):
# Guarda columnas de dataframe como dataframes, y crea diccionario con respectivas columnas 
    cols = df.columns
    l = []
    for i in range(0,len(cols)):
        l.append(df[[cols[i]]])
    return l

def split_row(df_col):
    for i in range(0,len(df_col)):
        df_col[i] = df_col[i][cabeceras[i][0]].str.split('\n', expand=True).stack().reset_index().rename(columns={0:cabeceras[i][0]}).loc[:, df_col[i].columns].reset_index(drop=True)
    return df_col

def save_as_csv(df, output_path, tipo, nombre, sep=';'):
    df.to_csv(output_path + '\\' + dt.now().strftime('%d%m%Y') + '_' + tipo + nombre + '.csv', sep=sep)

def clean_assets(df):
    '''Limpiar encabezados'''
    encabezado = df.iloc[0].str.replace('\n','')
    df = df.iloc[2:]
    df.columns = encabezado
    df = df.reset_index(drop=True)
    
    '''Separar activos corrientes y no corrientes'''
    global activos_corr
    global activos_no_corr
    
    index_ac = df[df['ACTIVOS']=='ACTIVOS NO CORRIENTES'].index.tolist()
    activos_corr = df.iloc[0:index_ac[0]]
    activos_corr = activos_corr.rename(columns={'ACTIVOS':'ACTIVOS CORRIENTES'})
    activos_no_corr = df.iloc[index_ac[0]+1:]
    activos_no_corr = activos_no_corr.rename(columns={'ACTIVOS':'ACTIVOS NO CORRIENTES'})
    activos_corr = activos_corr.drop(activos_corr[activos_corr['Nota N°']==''].index).reset_index(drop=True)
    activos_no_corr = activos_no_corr.drop(activos_no_corr[activos_no_corr['Nota N°']==''].index).reset_index(drop=True)
    
    '''Dejar numeros en formato'''
    global fecha1
    global fecha2
    
    fecha1 = encabezado[3]
    fecha2 = encabezado[4]
    activos_corr = num_format(activos_corr, fecha1)
    activos_corr = num_format(activos_corr, fecha2)
    activos_no_corr = num_format(activos_no_corr, fecha1)
    activos_no_corr = num_format(activos_no_corr, fecha2)
    
    '''Guardar DF como CSV'''
    tipo = 'ac'
    save_as_csv(activos_corr, output_path, tipo , nombre)
    tipo = 'anc'
    save_as_csv(activos_no_corr, output_path, tipo , nombre)
    
    return df

def clean_liability(df):
    '''Limpiar encabezados'''
    encabezado = df.iloc[0].str.replace('\n','')
    df = df.iloc[2:]
    df.columns = encabezado
    df = df.reset_index(drop=True)
    
    '''Separar pasivos corrientes, no corrientes y patrimonio'''
    global pasivos_corr
    global pasivos_no_corr
    global patrimonio
    
    index_pc = df[df['PASIVOS Y PATRIMONIO NETO']=='PASIVOS NO CORRIENTES'].index.tolist()
    index_pat = df[df['PASIVOS Y PATRIMONIO NETO']=='PATRIMONIO NETO'].index.tolist()
    
    pasivos_corr = df.iloc[0:index_pc[0]]
    pasivos_corr = pasivos_corr.rename(columns={'PASIVOS Y PATRIMONIO NETO':'PASIVOS CORRIENTES'})  
    pasivos_corr = pasivos_corr.drop(pasivos_corr[pasivos_corr['Nota N°']==''].index).reset_index(drop=True)
    
    pasivos_no_corr = df.iloc[index_pc[0]+1:index_pat[0]]
    pasivos_no_corr = pasivos_no_corr.rename(columns={'PASIVOS Y PATRIMONIO NETO':'PASIVOS NO CORRIENTES'})
    pasivos_no_corr = pasivos_no_corr.drop(pasivos_no_corr[pasivos_no_corr['Nota N°']==''].index).reset_index(drop=True)
    
    patrimonio = df.iloc[index_pat[0]+1:]
    patrimonio = patrimonio.rename(columns={'PASIVOS Y PATRIMONIO NETO':'PATRIMONIO NETO'})
    patrimonio = patrimonio.drop(patrimonio[patrimonio['PATRIMONIO NETO']=='SUBTOTAL PATRIMONIO'].index)
    patrimonio = patrimonio.drop(patrimonio[patrimonio['PATRIMONIO NETO']=='TOTAL PATRIMONIO NETO'].index)
    patrimonio = patrimonio.drop(patrimonio[patrimonio['PATRIMONIO NETO']=='TOTAL PASIVOS Y PATRIMONIO NETO'].index)
    
    '''Dejar numeros en formato'''
    global fecha1
    global fecha2
    
    fecha1 = encabezado[3]
    fecha2 = encabezado[4]
    
    pasivos_corr = num_format(pasivos_corr, fecha1)
    pasivos_corr = num_format(pasivos_corr, fecha2)

    pasivos_no_corr = num_format(pasivos_no_corr, fecha1)
    pasivos_no_corr = num_format(pasivos_no_corr, fecha2)

    patrimonio = num_format(patrimonio, fecha1)
    patrimonio = num_format(patrimonio, fecha2)
    
    '''Guardar DF como CSV'''
    tipo = 'pc'
    save_as_csv(pasivos_corr, output_path, tipo , nombre)
    tipo = 'pnc'
    save_as_csv(pasivos_no_corr, output_path, tipo , nombre)
    tipo = 'pat'
    save_as_csv(patrimonio, output_path, tipo , nombre)
    
    return df

def clean_eerr_ori_efe(df):
    '''Limpiar encabezados'''
    encabezado = df.iloc[0].str.replace('\n','')
    df = df.iloc[2:]
    df.columns = encabezado
    df = df.reset_index(drop=True)
    
    '''Dejar numeros en formato'''
    global fecha_1
    global fecha_2
    
    fecha_1 = encabezado[3]
    fecha_2 = encabezado[4]
    
    df = num_format(df, fecha_1)
    df = num_format(df, fecha_2)
    
    '''Guardar DF como CSV'''
    save_as_csv(df, output_path, tipo , nombre)
    
    return df

def clean_ecpn(df):
    '''Limpiar encabezados'''
    encabezado = df.iloc[1].str.replace('\n','').tolist()
    encabezado[0] = encabezado[0].replace('','Concepto')
    encabezado[-1] = encabezado[-1].replace('','Total')
    encabezado[-2] = encabezado[-2].replace('','Participaciones no controladoras')
    encabezado[-3] = encabezado[-3].replace('','Otros resultados integrales')
    df = df.iloc[2:]
    df.columns = encabezado
    df = df.reset_index(drop=True)
    
    '''Separar registros en filas'''
    # Separar cada columna en DF
    df_col = cols_in_df(df)
    df_col.pop()
    
    #Aplicar función para separar filas
    global cabeceras
    
    cabeceras = []
    for i in range(0,len(df_col)):
        cabeceras.append(list(df_col[i]))
    aux = split_row(df_col)
    
    #Eliminar registro que no sirve
    aux[0] = aux[0].drop(aux[0][aux[0][cabeceras[0][0]]=='Otras variaciones patrimoniales '].index).reset_index(drop=True)
    
    '''Dejar numeros en formato'''
    for i in range(1,len(aux)):
        aux[i] = num_format(aux[i], cabeceras[i][0])
        
    '''Reconstituir el DF'''
    df = pd.concat(aux, axis=1)
    df['Total'] = df.sum(axis=1)
    
    '''Guardar DF como CSV'''
    save_as_csv(df, output_path, tipo , nombre)
    
    return df

## Cargar PDF

Se cargan las páginas en donde se encuentran los estados financieros solamente. Los Estados Financieros en cada página son:
 - Estado de Situación Financiera (ESF) - Activos -> Página 2
 - Estado de Situación Financiera (ESF) - Pasivos y Pat. -> Página 3
 - Estado de Resultados (EERR) -> Página 4
 - Estado de Resultados Integral (ORI) -> Página 5
 - Estado de Cambios en el Patrimonio Neto (ECPN) -> Página 6 y 7
 - Estado de Flujo de Efectivo (EFE) -> Página 8

In [3]:
input_path = 'C:/Users/ISINIESCARC/Desktop/Proyecto_Track_2/Mineria_pdf/EEFF/'

In [4]:
pdfs = read_pdf(input_path)

In [5]:
keys = list(pdfs.keys())
global keys

## ESF - Activos

In [6]:
i = 0

for pdf in pdfs.keys():
    
    global output_path
    output_path = 'C:/Users/ISINIESCARC/Desktop/Proyecto_Track_2/Mineria_pdf/EEFF/output/activos/'
    
    global nombre
    nombre = keys[i]
   
    pdfs[pdf][0].df = clean_assets(pdfs[pdf][0].df)
    
    i = i + 1

## ESF - Pasivos

In [7]:
i = 0

for pdf in pdfs.keys():
    
    global output_path
    output_path = 'C:/Users/ISINIESCARC/Desktop/Proyecto_Track_2/Mineria_pdf/EEFF/output/pasivos/'
    
    global nombre
    nombre = keys[i]
   
    pdfs[pdf][1].df = clean_liability(pdfs[pdf][1].df)
    
    i = i + 1

## Estado de Resultados (EERR)

In [8]:
i = 0

for pdf in pdfs.keys():
    
    global output_path
    output_path = 'C:/Users/ISINIESCARC/Desktop/Proyecto_Track_2/Mineria_pdf/EEFF/output/eerr/'
    
    global tipo
    tipo = 'eerr'
    
    global nombre
    nombre = keys[i]
   
    pdfs[pdf][2].df = clean_eerr_ori_efe(pdfs[pdf][2].df)
    
    i = i + 1

## Otros Resultados Integrales (ORI)

In [9]:
i = 0

for pdf in pdfs.keys():
    
    global output_path
    output_path = 'C:/Users/ISINIESCARC/Desktop/Proyecto_Track_2/Mineria_pdf/EEFF/output/ori/'
    
    global tipo
    tipo = 'ori'
    
    global nombre
    nombre = keys[i]
   
    pdfs[pdf][3].df = clean_eerr_ori_efe(pdfs[pdf][3].df)
    
    i = i + 1

## ECPN periodo anterior

In [10]:
i = 0

for pdf in pdfs.keys():
    
    global output_path
    output_path = 'C:/Users/ISINIESCARC/Desktop/Proyecto_Track_2/Mineria_pdf/EEFF/output/ecpn/'
    
    global tipo
    tipo = 'per_anterior'
    
    global nombre
    nombre = keys[i]
   
    pdfs[pdf][4].df = clean_ecpn(pdfs[pdf][4].df)
    
    i = i + 1

## ECPN periodo actual

In [11]:
i = 0

for pdf in pdfs.keys():
    
    global output_path
    output_path = 'C:/Users/ISINIESCARC/Desktop/Proyecto_Track_2/Mineria_pdf/EEFF/output/ecpn/'
    
    global tipo
    tipo = 'per_actual'
    
    global nombre
    nombre = keys[i]
   
    pdfs[pdf][5].df = clean_ecpn(pdfs[pdf][5].df)
    
    i = i + 1

## Estado de Flujo de Efectivo (EFE)

In [12]:
i = 0

for pdf in pdfs.keys():
    
    global output_path
    output_path = 'C:/Users/ISINIESCARC/Desktop/Proyecto_Track_2/Mineria_pdf/EEFF/output/efe/'
    
    global tipo
    tipo = 'efe'
    
    global nombre
    nombre = keys[i]
   
    pdfs[pdf][6].df = clean_eerr_ori_efe(pdfs[pdf][6].df)
    
    i = i + 1